# Run An Mendelian Randomization

### Authors: Calvin Howard.

#### Last updated: July 6, 2023

Use this to run/test a statistical model on a spreadsheet.

Notes:
- To best use this notebook, you must understand MR and its 3 critical assumptions. 

# 00 - Import CSV with All Data
**The CSV is expected to be in this format**
- ID and absolute paths to niftis are critical
```
+-----+----------------------------+--------------+--------------+--------------+
| ID  | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

Prep Output Direction

In [ ]:
# Specify where you want to save your results to
out_dir = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Research/2023/subiculum_cognition_and_age/figures/Figures/odds_ratios'

Import Data

In [ ]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Dropbox (Partners HealthCare)/studies/cognition_2023/metadata/master_list_proper_subjects.xlsx'
sheet = 'master_list_proper_subjects'

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()

# 01 - Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [ ]:
data_df.columns

In [ ]:
drop_list = ['City', 'StimMatch', 'Cognitive_Improve', 'Age']

In [ ]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)
display(data_df)

**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

In [ ]:
data_df.columns

Set the parameters for dropping rows

In [ ]:
column = 'City'  # The column you'd like to evaluate
condition = 'not'  # The condition to check ('equal', 'above', 'below', 'not')
value = 'Wurzburg' # The value to drop if found

In [ ]:
data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
display(data_df)

**Standardize Data**
- Enter Columns you Don't want to standardize into a list

In [ ]:
# Remove anything you don't want to standardize
cols_not_to_standardize = None # ['Z_Scored_Percent_Cognitive_Improvement_By_Origin_Group', 'Z_Scored_Subiculum_T_By_Origin_Group_'] #['Age']

In [ ]:
data_df = cal_palm.standardize_columns(cols_not_to_standardize)
data_df

In [ ]:
# for col in data_df.columns:
#     if 'CSF' and 'eh' not in col:
#         data_df[col] = data_df[col] * -1

RCT Plotter

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tools import add_constant
import patsy

class MendelianRandomizationAnalysis:
    def __init__(self, data, outcome_cols, exposure_col, iv_col, covariate_cols=None):
        """
        Initializes the class with the data and variable specifications.
        """
        self.data = data
        self.outcome_cols = outcome_cols
        self.exposure_col = exposure_col
        self.iv_col = iv_col
        self.covariate_cols = covariate_cols if covariate_cols else []

    def label_encode_variables(self):
        """
        Label encode categorical variables if needed.
        """
        categorical_columns = self.data.select_dtypes(include=['object', 'category']).columns
        for col in categorical_columns:
            self.data[col] = pd.factorize(self.data[col])[0]

    def run_regression(self, y_col, X_cols):
        """
        Runs a regression and returns the model.
        """
        X = add_constant(self.data[X_cols])  # Add a constant term for the intercept
        y = self.data[y_col]
        model = sm.OLS(y, X).fit()
        return model

    def first_stage_regression(self):
        """
        First-stage regression of the IV analysis: regress exposure on IV.
        """
        return self.run_regression(self.exposure_col, [self.iv_col] + self.covariate_cols)

    def second_stage_regression(self, outcome_col, predicted_exposure):
        """
        Second-stage regression of the IV analysis: regress outcome on the predicted exposure.
        """
        return self.run_regression(outcome_col, [predicted_exposure] + self.covariate_cols)

    def mendelian_randomization(self):
        """
        Executes the two-stage least squares Mendelian Randomization.
        """
        self.label_encode_variables()
        first_stage_model = self.first_stage_regression()
        self.data['predicted_exposure'] = first_stage_model.predict()

        results = {}
        for outcome in self.outcome_cols:
            second_stage_model = self.second_stage_regression(outcome, 'predicted_exposure')
            results[outcome] = second_stage_model

        return results

In [ ]:
data_df.columns

In [ ]:
# Usage example:
# Assuming 'df' is a pandas DataFrame with the necessary data.
mr_analysis = MendelianRandomizationAnalysis(
    data=data_df,
    outcome_cols=['Percent_Cognitive_Improvement'],
    exposure_col='StimMatch',
    iv_col='Age',
    covariate_cols=None
)

mr_results = mr_analysis.mendelian_randomization()
for outcome, model in mr_results.items():
    print(f'Results for {outcome}:')
    print(model.summary())
